In [16]:
import numpy as np 
import pandas as pd
import os
import torch
import torchvision
from torchvision import datasets
import torchvision.transforms as transforms
from torch import nn, optim
from torch.nn import functional as F
from torch.utils.data import DataLoader,Dataset, sampler, random_split
from torchvision import models
!pip install timm # kaggle doesnt have it installed by default
import timm
from timm.loss import LabelSmoothingCrossEntropy # This is better than normal nn.CrossEntropyLoss
import matplotlib.pyplot as plt
%matplotlib inline
import sys
from tqdm import tqdm
import time
import copy
from os import listdir
from os.path import isfile, join
from PIL import Image
import glob 
import math
from typing import Any, Callable, cast, Dict, List, Optional, Tuple, Union


In [17]:
#Método para obtener todas las especies de pájaros a partir de la estructura de carpetas
def get_classes(data_dir):
    all_data = datasets.ImageFolder(data_dir)
    return all_data.classes

In [18]:
#Método para entrenar el modelo
def train_model(model, criterion, optimizer, scheduler, dataloaders, dataset_sizes, num_epochs=10):
    since = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    
    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print("-"*10)
        
        for phase in ['train', 'val']: 
            if phase == 'train':
                model.train() # model to training mode
            else:
                model.eval() # model to evaluate
            
            running_loss = 0.0
            running_corrects = 0.0
            
            for inputs, labels in tqdm(dataloaders[phase]):
                inputs = inputs.to(device)
                labels = labels.to(device)
                
                optimizer.zero_grad()
                
                with torch.set_grad_enabled(phase == 'train'): # no autograd makes validation go faster
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1) # used for accuracy
                    loss = criterion(outputs, labels)
                    
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                
            if phase == 'train':
                scheduler.step() # step at end of epoch
            
            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc =  running_corrects.double() / dataset_sizes[phase]
            
            print("{} Loss: {:.4f} Acc: {:.4f}".format(phase, epoch_loss, epoch_acc))
            
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict()) # keep the best validation accuracy model
        print()

    time_elapsed = time.time() - since # slight error
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print("Best Val Acc: {:.4f}".format(best_acc))
    
    model.load_state_dict(best_model_wts)
    return model

In [19]:
def pre_image(image_path,model):
    #img = Image.open(image_path)
    #transform_norm = transforms.Compose([transforms.ToTensor(), 
    #transforms.Resize((224,224))])
    transform_norm = transforms.ToTensor()
   # get normalized image
    img_normalized = transform_norm(image_path).float()
    img_normalized = img_normalized.unsqueeze(0)
   # input = Variable(image_tensor)
    img_normalized = img_normalized.to(device)
   # print(img_normalized.shape)
    with torch.no_grad():
        model.eval()
        output =model(img_normalized)
     # print(output)
        index = output.data.cpu().numpy().argmax()
        class_name = classes[index]
        return class_name

In [20]:
IMG_EXTENSIONS = (".jpg", ".jpeg", ".png", ".ppm", ".bmp", ".pgm", ".tif", ".tiff", ".webp")

class ImageFolderCustom(datasets.DatasetFolder):
    
    def __init__(
        self,
        root: str,
        setname: str,
        transform: Optional[Callable] = None,
        target_transform: Optional[Callable] = None,
        loader: Callable[[str], Any] = datasets.folder.default_loader,
        is_valid_file: Optional[Callable[[str], bool]] = None,
    ):
        super().__init__(
            root,
            loader,
            IMG_EXTENSIONS if is_valid_file is None else None,
            transform=transform,
            target_transform=target_transform,
            is_valid_file=is_valid_file,
        )
        
        classes, class_to_idx = self.find_classes(self.root)
        self.samples = self.make_dataset2(self.root,setname, class_to_idx, IMG_EXTENSIONS, is_valid_file)
        self.imgs = self.samples
        
    @staticmethod
    def make_dataset2(
        directory: str,
        setname: str,
        class_to_idx: Optional[Dict[str, int]] = None,
        extensions: Optional[Union[str, Tuple[str, ...]]] = None,
        is_valid_file: Optional[Callable[[str], bool]] = None,    
    ) -> List[Tuple[str, int]]:
        
        setname = setname
        assert setname in ['train','val']

        if class_to_idx is None:
            _, class_to_idx = find_classes(directory)
        elif not class_to_idx:
            raise ValueError("'class_to_index' must have at least one entry to collect any samples.")

        both_none = extensions is None and is_valid_file is None
        both_something = extensions is not None and is_valid_file is not None
        if both_none or both_something:
            raise ValueError("Both extensions and is_valid_file cannot be None or not None at the same time")

        if extensions is not None:

            def is_valid_file(x: str) -> bool:
                return datasets.folder.has_file_allowed_extension(x, extensions)  # type: ignore[arg-type]

        is_valid_file = cast(Callable[[str], bool], is_valid_file)

        instances = []
        available_classes = set()
        for target_class in sorted(class_to_idx.keys()):
            class_index = class_to_idx[target_class]
            target_dir = os.path.join(directory, target_class)
            if not os.path.isdir(target_dir):
                continue
            for root, _, fnames in sorted(os.walk(target_dir, followlinks=True)):
                num_images=len(fnames)
                num_separator=math.ceil(num_images*0.9)
                i=0
                #print(i, num_separator)
                #print(setname=='train' and i>=0 and i<num_separator)
                for fname in sorted(fnames):
                    path = os.path.join(root, fname)
                    #print(i)
                    if(setname=='train' and i>=0 and i<num_separator or (setname=='val' and i>=num_separator and i<num_images)):
                        if is_valid_file(path):
                            item = path, class_index
                            #print(item)
                            instances.append(item)
                            if target_class not in available_classes:
                                available_classes.add(target_class)
                    i=i+1
    
        empty_classes = set(class_to_idx.keys()) - available_classes
        if empty_classes:
            msg = f"Found no valid file for the classes {', '.join(sorted(empty_classes))}. "
            if extensions is not None:
                msg += f"Supported extensions are: {extensions if isinstance(extensions, str) else ', '.join(extensions)}"
            raise FileNotFoundError(msg)
    
        return instances

In [21]:
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self, dropout=0.0):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16*53*53,8000)
        self.dropout = nn.Dropout(dropout)
        self.fc2 = nn.Linear(8000, 2000)
        self.fc3 = nn.Linear(2000, 400)

    def forward(self, x):
        
        x = self.pool(F.relu(self.conv1(x)))
        
        x = self.pool(F.relu(self.conv2(x)))
        
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        
        x = F.relu(self.fc1(x))
        
        x = F.relu(self.fc2(x))
        
        x = self.fc3(x)
        
        return x

In [22]:

 #train
transform_train = transforms.Compose([
    #transforms.RandomHorizontalFlip(),
    #transforms.RandomVerticalFlip(),
    #transforms.RandomApply(torch.nn.ModuleList([transforms.ColorJitter()]), p=0.25),
    #transforms.Resize(256),
    #transforms.CenterCrop(224),
    transforms.ToTensor(),
    #transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)), # imagenet means
    #transforms.RandomErasing(p=0.2, value='random')
])
transform_val = transforms.Compose([
    #transforms.Resize(256),
    #transforms.CenterCrop(224),
    transforms.ToTensor(),
    #transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
])
    
train_data = ImageFolderCustom(root='../input/iais22-birds/birds/birds',setname='train', transform = transform_train)
train_loader = DataLoader(train_data, batch_size=256, shuffle=True, num_workers=4)
train_data_len = len(train_data)

val_data = ImageFolderCustom(root='../input/iais22-birds/birds/birds',setname='val', transform = transform_val)
val_loader = DataLoader(val_data, batch_size=64, shuffle=True, num_workers=4)
valid_data_len = len(val_data)

print(f"Found {len(train_data)} images for training with {len(train_data.classes)} classes")
print(f"Found {len(val_data)} images for validation with {len(val_data.classes)} classes")

final_train_data = datasets.ImageFolder(root='../input/iais22-birds/birds/birds', transform = transform_train)
final_train_loader = DataLoader(final_train_data, batch_size=256, shuffle=True, num_workers=4)
final_train_data_len = len(final_train_data)

print(f"Found {len(final_train_data)} images for final training with {len(final_train_data.classes)} classes")

/opt/conda/lib/python3.7/site-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Found 52714 images for training with 400 classes
Found 5674 images for validation with 400 classes
Found 58388 images for final training with 400 classes


In [23]:
dataloaders = {
    "train": train_loader,
    "val": val_loader
}
dataset_sizes = {
    "train": train_data_len,
    "val": valid_data_len
}

final_dataloaders = {
    "train": final_train_loader,
    "val": val_loader
}
final_dataset_sizes = {
    "train": final_train_data_len,
    "val": valid_data_len
}

In [24]:
classes = get_classes("../input/iais22-birds/birds/birds")

In [25]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

torch.backends.cudnn.benchmark = True


model = Net(dropout=0.2).to(device)


In [26]:
criterion = nn.CrossEntropyLoss(label_smoothing=0.11)
criterion = criterion.to(device)
optimizer = optim.Adam(model.parameters(), lr=0.0003)

In [27]:
exp_lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.97)

In [28]:
#model_ft = train_model(model, criterion, optimizer, exp_lr_scheduler, dataloaders, dataset_sizes)

In [29]:
model_ft = train_model(model, criterion, optimizer, exp_lr_scheduler, final_dataloaders, final_dataset_sizes,num_epochs=10)

Epoch 0/9
----------


100%|██████████| 229/229 [02:02<00:00,  1.87it/s]


train Loss: 5.5809 Acc: 0.0260


100%|██████████| 89/89 [00:12<00:00,  7.25it/s]


val Loss: 5.0094 Acc: 0.0753

Epoch 1/9
----------


...

val Loss: 1.8481 Acc: 0.8650

Epoch 9/9
----------


100%|██████████| 229/229 [02:01<00:00,  1.89it/s]


train Loss: 1.8060 Acc: 0.8758


100%|██████████| 89/89 [00:11<00:00,  7.49it/s]

val Loss: 1.6355 Acc: 0.9392

Training complete in 22m 19s
Best Val Acc: 0.9392


In [30]:

image_list = []
preds_id = []
for filename in glob.glob("../input/iais22-birds/submission_test/submission_test/*.jpg"): 
    im=Image.open(filename)
    id = os.path.basename(filename).split(".")[0]
    image_list.append(im)
    preds_id.append(id)

index=[]
preds = []
for f in image_list:
    i = image_list.index(f)+1
    predict_class = pre_image(f,model)
    index.append(i)
    preds.append(predict_class)
    if(i%500==0):
        print(i)

submission = pd.DataFrame(
    data =np.array([preds_id,preds ]).T, 
    columns = ["Id", "Category"]
)
submission.to_csv("submission.csv", index = False)
submission.head()

500
1000
1500
2000


,Id,Category
0,8421278,MAGPIE GOOSE
1,9541931,WILD TURKEY
2,1820866,FLAME BOWERBIRD
3,12992,CAPE LONGCLAW
4,12761483,PELICAN
